In [1]:
import numpy as np
import pandas as pd
import tqdm
import jax
import jax.numpy as jnp
import string
import tensorflow as tf
import time
from tqdm import tqdm
import nltk
from nltk.corpus import stopwords

## Data Preprocessing

In [2]:
data = pd.read_csv('./data/raw data/raw_data.csv', header=0, names=['text'], usecols=[1])
print(f'Data Shape: {data.shape}')
data.head()

Data Shape: (13368, 1)


,text
0,"Sally Forrest, an actress-dancer who graced th..."
1,A middle-school teacher in China has inked hun...
2,A man convicted of killing the father and sist...
3,Avid rugby fan Prince Harry could barely watch...
4,A Triple M Radio producer has been inundated w...


In [3]:
# remove punctuation
punctuations = string.punctuation
def remove_punctuation(txt):
    for char in punctuations:
        if char in txt:
            txt = txt.replace(char, "")
    return txt

# change to lower caps
data['text'] = data['text'].str.lower()

# remove punctuations
data['text'] = data['text'].apply(remove_punctuation)

In [4]:
# remove stopwords
# read stopwords from data/raw data/stopwords.txt
stop_words = []
with open('./data/raw data/stopwords.txt', 'r') as f:
    for line in f:
        stop_words.append(line.strip())

def remove_stopwords(txt):
    txt = [word for word in txt.split() if word not in stop_words]
    return ' '.join(txt)

data['text'] = data['text'].apply(remove_stopwords)

In [5]:
# split each row into list of words
data_lst = data['text'].apply(lambda txt: txt.split(" "))

# select number of rows to be used as training data
nrows = 200
random_indices = np.random.randint(low=0, high=len(data_lst), size=nrows)
data_lst = data_lst[random_indices].reset_index(drop=True)

print(f'Number of rows of data: {len(data_lst)}')
data_lst[:5]

Number of rows of data: 200


0    [cnnafter, handful, events, months, hillary, c...
1    [account, manager, quit, job, professional, vl...
2    [missing, philae, space, probe, bumped, onto, ...
3    [cnnigniting, live, cage, severing, heads, doz...
4    [final, scoreline, suggested, otherwise, gordo...
Name: text, dtype: object

In [6]:
# vocab dict
vocab, index = {}, 1
vocab['<pad>'] = 0
for line in data_lst:
    for word in line:
        if word not in vocab:
            vocab[word] = index
            index += 1

# inverse_vocab dict
inverse_vocab = {}
for word, index in vocab.items():
    inverse_vocab[index] = word

print(f'Vocab size: {len(vocab)}')

Vocab size: 14519


In [7]:
# sequences
sequences = []
for line in data_lst:
    vectorized_line = [vocab[word] for word in line]
    sequences.append(vectorized_line)

## Split Train and Test Sets

In [8]:
# split into train and test sets
# choose 20 random sequences
ntest = 20
test_indices = np.random.randint(low=0, high=len(sequences), size=ntest)
test_sequences = [sequences[i] for i in test_indices]
train_sequences = [sequences[i] for i in range(len(sequences)) if i not in test_indices]

In [9]:
# function to generate samples
def generate_training_data(sequences, window_size, num_ns, vocab_size, seed):
  # Elements of each training example are appended to these lists.
  targets, contexts, labels = [], [], []

  # Build the sampling table for `vocab_size` tokens.
  sampling_table = tf.keras.preprocessing.sequence.make_sampling_table(vocab_size)

  # Iterate over all sequences (sentences) in the dataset.
  for sequence in tqdm(sequences):

    # Generate positive skip-gram pairs for a sequence (sentence).
    positive_skip_grams, _ = tf.keras.preprocessing.sequence.skipgrams(
          sequence,
          vocabulary_size=vocab_size,
          sampling_table=sampling_table,
          window_size=window_size,
          negative_samples=0,
          shuffle=True)

    # Iterate over each positive skip-gram pair to produce training examples
    # with a positive context word and negative samples.
    for target_word, context_word in positive_skip_grams:
      context_class = tf.reshape(tf.constant([context_word], dtype="int64"), (1,1))
      negative_sampling_candidates, _, _ = tf.random.log_uniform_candidate_sampler(
          true_classes=context_class,
          num_true=1,
          num_sampled=num_ns,
          unique=True,
          range_max=vocab_size,
          seed=seed,
          name="negative_sampling")

      # Build context and label vectors (for one target word)
      context = tf.concat([tf.squeeze(context_class,1), negative_sampling_candidates], 0)
      label = tf.constant([1] + [0]*num_ns, dtype="int64")

      # Append each element from the training example to global lists.
      targets.append(target_word)
      contexts.append(context)
      labels.append(label)

  return targets, contexts, labels

In [10]:
# function to generate testing data
def generate_testing_data(sequences, vocab_size, window_size):
    targets, contexts, labels = [], [], []
    for sequence in tqdm(sequences):
        positive_skip_grams, _ = tf.keras.preprocessing.sequence.skipgrams(
            sequence,
            vocabulary_size=vocab_size,
            window_size=window_size,
            negative_samples=0)
    for target_word, context_word in positive_skip_grams:
        targets.append(target_word)
        contexts.append(context_word)
        labels.append(1)
    return targets, contexts, labels

In [11]:
# generate training data
window_size = 5
num_ns = 4
vocab_size = len(vocab)
seed = 4212

targets, contexts, labels = generate_training_data(sequences=train_sequences,
                                                 window_size=window_size,
                                                 num_ns=num_ns,
                                                 vocab_size=vocab_size,
                                                 seed=seed)

targets = np.array(targets)
contexts = np.array(contexts)
labels = np.array(labels)

print(f'targets shape: {targets.shape}')
print(f'contexts shape: {contexts.shape}')
print(f'labels shape: {labels.shape}')

100%|██████████| 183/183 [02:29<00:00,  1.23it/s]


targets shape: (309499,)
contexts shape: (309499, 5)
labels shape: (309499, 5)


In [12]:
# generate testing data
targets_test, contexts_test, labels_test = generate_testing_data(sequences=test_sequences,
                                                                    vocab_size=vocab_size,
                                                                    window_size=window_size)

targets_test = np.array(targets_test)
contexts_test = np.array(contexts_test)
labels_test = np.array(labels_test)

print(f'targets_test shape: {targets_test.shape}')
print(f'contexts_test shape: {contexts_test.shape}')
print(f'labels_test shape: {labels_test.shape}')

100%|██████████| 20/20 [00:00<00:00, 158.04it/s]

targets_test shape: (3970,)
contexts_test shape: (3970,)
labels_test shape: (3970,)


**Sanity Check on quality of training and testing data**

In [13]:
# training data
print(f"target_index    : {targets[0]}")
print(f"target_word     : {inverse_vocab[targets[0]]}")
print(f"context_indices : {contexts[0]}")
print(f"context_words   : {[inverse_vocab[c] for c in contexts[0]]}")
print(f"label           : {labels[0]}")

print("target  :", targets[0])
print("context :", contexts[0])
print("label   :", labels[0])

target_index    : 120
target_word     : irish
context_indices : [ 128   38 1557 1567 1050]
context_words   : ['welsh', 'gala', 'portuguese', 'prodded', 'wee']
label           : [1 0 0 0 0]
target  : 120
context : [ 128   38 1557 1567 1050]
label   : [1 0 0 0 0]


In [14]:
# testing data
print(f"target_index    : {targets_test[0]}")
print(f"target_word     : {inverse_vocab[targets_test[0]]}")
print(f"context_index : {contexts_test[0]}")
print(f"context_word   : {inverse_vocab[contexts_test[0]]}")
print(f"label           : {labels_test[0]}")

print("target  :", targets_test[0])
print("context :", contexts_test[0])
print("label   :", labels_test[0])

target_index    : 709
target_word     : health
context_index : 14466
context_word   : cadwaladr
label           : 1
target  : 709
context : 14466
label   : 1


## Minimizing Objective Function for SGNS

$$

\min_{\theta} = \frac{1}{N} \sum_{i=1}^{N} [log \sigma(u_{ic}^T)  + \sum_{k=1}^{K}log \sigma(-u_{kc}^T v_{iw})]

\\
\\
\theta = [U, V]

$$

In [51]:
# sigmoid function
def sigmoid(x):
    """Inputs a real number, outputs a real number"""
    return 1 / (1 + jnp.exp(-x))


In [52]:
# define function to get embedding vectors
def get_embedding_vectors(target, context, V, U):
    """
    Input (example)
    target = (188,)
    context = (93, 40, 1648, 1659, 1109)
    V: matrix of dim (n x |v|)
    U: matrix of dim (|v| x n)
        n = embedding dimension, |v| = vocab size

    Output
    v_t: target word vector, dimension: (n,)
    u_c: context word vectors, consists of u_pos and u_neg: dimension: (n, len(context))
    """
    target = target.astype(int)
    context = context.astype(int)
    v_t = V.T[target]
    u_c = U[context]
    return v_t, u_c

# t = targets[0]
# c = contexts[0]
# v_test, u_test = get_embedding_vectors(t, c, V, U)
# print(f'v_test shape: {v_test.shape}')
# print(f'u_test shape: {u_test.shape}')

In [53]:
# define local_loss function
@jax.jit
def local_loss(params):
    """
    Input (example)
    params = [v_t, jnp.array([u_pos, u_neg])]
        v_t: target word vector, dimension: (n,)
        u_c: context word vectors, consists of u_pos and u_neg: dimension: (len(context), n)

    Output
    local_loss: real number
    """
    v_t = params[0]
    u_c = params[1]
    return -jnp.log(sigmoid(jnp.dot(u_c[0], v_t))) - jnp.sum(jnp.log(sigmoid(-jnp.dot(u_c[1:], v_t))))

# p = [V[:, 0], U[:5, :]]
# print(f'local_loss: {local_loss(p)}')


In [54]:
# define gradient function
L_grad = jax.grad(local_loss)

# g = L_grad([V[:, 0], U[:5, :]])
# print(f"g[0] shape: {g[0].shape}")
# print(f"g[1] shape: {g[1].shape}")

In [65]:
# define a function to update parameters
# def update_params(target, context, partial_v, partial_u, V, U):
#     t = target; print(f'target: {type(t)}')
#     c = context; print(f'context: {type(c)}')
#     V[:, t] = V[:, t] - lr * partial_v
#     U[c, :] = U[c, :] - lr * partial_u

def update_params(target, context, partial_v, partial_u, V, U):
    # Update V and U
    V_updated = V.at[:, target].add(-lr * partial_v)
    U_updated = U.at[context, :].add(-lr * partial_u)

    return V_updated, U_updated

In [66]:
# define batch functions
get_batch_embedding_vectors = jax.vmap(get_embedding_vectors, in_axes=(0, 0, None, None))

batch_losses = jax.vmap(local_loss, in_axes=(0))

batch_grads = jax.vmap(L_grad, in_axes=(0))

batch_update_params = jax.vmap(update_params, in_axes=(0, 0, 0, 0, None, None))

In [67]:
# set up
n = 300
v = len(vocab)
V = jnp.array(np.random.normal(0, 1, size=(n, v)) / np.sqrt(v))
U = jnp.array(np.random.normal(0, 1, size=(v, n)) / np.sqrt(v))

params = [V, U]
targets_data = jnp.array(targets.astype(float))
contexts_data = jnp.array(contexts.astype(float))
labels_data = jnp.array(labels.astype(float))

print(f'V shape: {V.shape}')
print(f'U shape: {U.shape}')
print(f'targets_data shape: {targets_data.shape} \t type: {type(targets_data).__name__}')
print(f'contexts_data shape: {contexts_data.shape} \t type: {type(contexts_data).__name__}')
print(f'labels_data shape: {labels_data.shape} \t type: {type(labels_data).__name__}')

V shape: (300, 14519)
U shape: (14519, 300)
targets_data shape: (309499,) 	 type: ArrayImpl
contexts_data shape: (309499, 5) 	 type: ArrayImpl
labels_data shape: (309499, 5) 	 type: ArrayImpl


In [68]:
# train using stochastic gradient descent

# set up
N = len(targets_data)
lr = 1.
n_epochs = 5
batch_size = 1000
n_batches = N // batch_size

epoch_losses = []

# gradient descent
for epoch in range(n_epochs):
    start_time = time.time()

    # shuffle data
    perm = np.random.permutation(N)
    targets_epoch = targets_data[perm]
    contexts_epoch = contexts_data[perm]
    labels_epoch = labels_data[perm]

    # decrease learning rate
    if epoch % 2 == 0:
        lr /= 2

    # losses in each epoch
    losses = []
    for batch in tqdm(range(n_batches)):
        targets_batch = targets_epoch[batch*batch_size: (batch+1)*batch_size]
        contexts_batch = contexts_epoch[batch*batch_size: (batch+1)*batch_size]
        labels_batch = labels_epoch[batch*batch_size: (batch+1)*batch_size]

        # get batch embedding vectors
        v_t_batch, u_c_batch = get_batch_embedding_vectors(targets_batch, contexts_batch, V, U)

        # calculate batch_losses and batch_grads
        loss_values = np.mean(batch_losses([v_t_batch, u_c_batch]))
        grads = batch_grads([v_t_batch, u_c_batch])

        # batch update parameters
        partial_V_batch = grads[0]
        partial_U_batch = grads[1]

        batch_update_params(targets_batch, contexts_batch, partial_V_batch, partial_U_batch, V, U)

        # store the loss value
        losses.append(np.mean(loss_values))
    
    # store epoch losses
    epoch_losses.append(np.mean(losses))

    end_time = time.time()
    
    # print epoch loss
    if epoch % 1 == 0:
        print(f"Epoch {epoch+1}/{n_epochs} \t loss = {np.mean(epoch_losses)} \t time = {end_time - start_time:.2f}s")

  0%|          | 0/309 [00:00<?, ?it/s]

  0%|          | 0/309 [00:00<?, ?it/s]


TypeError: Indexer must have integer or boolean type, got indexer with type float32 at position 1, indexer value Traced<ShapedArray(float32[])>with<BatchTrace(level=1/0)> with
  val = Array([ 6837.,  9358.,  8486., 13757.,  6831.,  3024.,   976.,   648.,
       10064.,  9505.,   296.,  7598., 11652.,  3101.,  9277., 12584.,
       11340.,  1044.,  8537.,  1257., 10175.,  6819.,  6354.,  6369.,
        3074.,   684.,  5253.,  4097., 13407.,  1133., 12121.,  9409.,
       14395.,  3654.,  4420.,  8502.,   846.,  7232.,  2572., 11862.,
        1541.,  6562., 14391.,  9669., 10520.,  2263.,  1339.,  7096.,
        1834.,  4628.,  6641.,  5897.,  1284., 11829.,  6472.,  8953.,
        4005.,  4673.,  7956.,  2556.,  5122.,  3722.,  1313., 11397.,
        4021.,  2469., 13226.,  1468.,   888.,  1902.,  5171.,  1689.,
        6466.,  8192.,  3573.,  2815.,  5617., 10437.,  3110.,  7802.,
       11910.,  2488.,  8364., 11290.,   902., 14353.,  9152.,  1481.,
        4397.,   316.,  7635.,  706

In [ ]:
# train using gradient descent

# set up
N = len(targets_data)
n_epochs = 5

lr = 1.

epoch_losses = []

# gradient descent
for epoch in range(n_epochs):

    start_time = time.time()

    # shuffle data
    perm = np.random.permutation(N)
    targets = targets_data[perm]
    contexts = contexts_data[perm]
    labels = labels_data[perm]

    losses = []
    for i in tqdm(range(N)):
        # stop after 1000 iterations
        if i == 1000:
            break
        target = targets[i].astype(int) # ; print(f'target shape: {target.shape}')
        context = contexts[i].astype(int)# ; print(f'context shape: {context}')
        label = labels[i].astype(int)

        # get the embedding vectors
        v_t, u_c = get_embedding_vectors(target, context, V, U)

        # get the loss value and gradient
        loss_value = local_loss([v_t, u_c])
        grad = L_grad([v_t, u_c])

        # update the parameters
        partial_V = grad[0]
        partial_U = grad[1]
        
        V[:, target] = V[:, target] - lr * partial_V
        U[context, :] = U[context, :] - lr * partial_U

        # store the loss value
        losses.append(loss_value)
    
    # store epoch losses
    epoch_losses.append(np.mean(losses))

    end_time = time.time()
    
    # print epoch loss
    if epoch % 1 == 0:
        print(f"Epoch {epoch+1}/{n_epochs} \t loss = {np.mean(epoch_losses)} \t time = {end_time - start_time:.2f}s")

In [ ]:
# # train using stochastic gradient descent

# # number of training examples
# N = len(targets_data)

# # learning rate
# lr = 3.

# # number of epochs
# n_epochs = 100

# # batch size
# batch_size = 10000

# # number of batches per epoch
# n_batches = N // batch_size

# # keep track of losses
# epoch_losses = []

# # training the network
# for epoch in range(n_epochs):
#     start_time = time.time()
#     # shuffle data
#     perm = np.random.permutation(N)
#     targets = targets[perm]
#     contexts = contexts[perm]
#     labels = labels[perm]

#     # half the learning rate every 25 epochs
#     if epoch == 50 or epoch == 75:
#         lr /= 2.

#     # losses in each epoch
#     losses = []
#     for batch in range(n_batches):
#         targets_batch = targets_data[batch*batch_size: (batch+1)*batch_size]
#         contexts_batch = contexts_data[batch*batch_size: (batch+1)*batch_size]
#         labels_batch = labels_data[batch*batch_size: (batch+1)*batch_size]

#         # calculate and save losses
#         loss_value, gradient = loss_value_and_grad(params, targets_batch, contexts_batch)
#         losses.append(loss_value)

#         params = [(V - lr*dV, U - lr*dU) for (V, U), (dV, dU) in zip(params, gradient)]

#     epoch_losses.append(np.mean(losses))

#     end_time = time.time()
#     if epoch % 1 == 0:
#         print(f"Epoch {epoch+1}/{n_epochs} \t loss = {np.mean(epoch_losses)} \t time = {end_time - start_time:.2f}s")

In [ ]:
# plot losses
import matplotlib.pyplot as plt
plt.plot(epoch_losses)
plt.xlabel('Iterations')
plt.ylabel('Loss')

In [ ]:
# V_ = np.copy(params[0][0])
# U_ = np.copy(params[0][1])

# # check dimensions of U and V, 10 epoch, 1r = 3, 1.5, 0.75, batch size = 500
# print(f'V_ shape: {V_.shape}')
# print(f'U_ shape: {U_.shape}')

In [ ]:
# copy U and V
V_trained = np.copy(params[0][0])
U_trained = np.copy(params[0][1])

# check dimensions of U and V, 100 epochs, lr = 1, batch_size = 500
print(f'V_trained shape: {V_trained.shape}')
print(f'U_trained shape: {U_trained.shape}')

## Evaluate against test set

In [ ]:
# define function that takes in an index and vocab size and returns the one-hot encoding
def getOneHot(index, vocab_size):
    onehot = np.zeros(vocab_size)
    onehot[index] = 1
    return onehot

# define softmax function
def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    return np.exp(x) / np.sum(np.exp(x), axis=0)

# test getOneHot and softmax function
print(getOneHot(1, 10))
print(softmax(np.array([1, 2, 3])))


In [ ]:
# check dimensions of U and V
print(f'U shape: {U.shape}')
print(f'V shape: {V.shape}')

In [ ]:
# see first 20 words in the vocab
test_words = list(vocab.keys())[20:40]
test_words

In [ ]:
# define cosine similarity scores between 2 word vectors
def similarity_score(target_word_embedding, context_word_embedding):
    return np.dot(target_word_embedding, context_word_embedding) / (np.linalg.norm(target_word_embedding) * np.linalg.norm(context_word_embedding))

# define a function that find the most similar words to a given word
def most_similar_words(word, V, n=5):
    scores = []
    target_word_idx = vocab[word]
    for i in range(V.shape[1]):
        if i == target_word_idx or inverse_vocab[i] == '<pad>':
            continue
        scores.append((inverse_vocab[i], similarity_score(V[:, target_word_idx], V[:, i])))
    scores = sorted(scores, key=lambda x: x[1], reverse=True)
    return scores[:n]

In [ ]:
# check similarity between words
# word: photos
print(most_similar_words('amazed', V_trained))

In [ ]:
# compute a forward pass through the skip-gram model

# define the forward pass function
def net(V, U, target_word_idx):
    target_hot = getOneHot(target_word_idx, len(vocab))
    return softmax( U @ V @ target_hot )

def predict(word, V, U):
    target_word_idx = vocab[word]
    y_hat = net(V, U, target_word_idx)
    # y_hat is the probability distribution over the vocab
    # select the top 5 words with the highest probability
    top_5 = np.argsort(y_hat)[-10:][::-1]
    top_5_words = [inverse_vocab[i] for i in top_5]
    return top_5_words

In [ ]:
# randomly select 1 word from vocab
word = np.random.choice(list(vocab.keys()))
y_hat = net(V_trained, U_trained, vocab[word])
print(f'Word: {word}')
predict(word, V_trained, U_trained)